In [761]:
from pymongo import MongoClient
import pymongo
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import re
import math

In [762]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [763]:
#compMoney = db.companies.find({'$and':[{"total_money_raised":{"$regex":".*M.*"}},{"founded_year":{"$gte":2000}}]},{'name':1,'total_money_raised':1,'offices':1,'_id':0}).sort("total_money_raised",-1)
#df = pd.DataFrame(compMoney)

compMoney = db.companies.find({"total_money_raised":{"$regex":".*M.*"}},{'name':1,'total_money_raised':1,'offices':1,'_id':0}).sort("total_money_raised",-1)
df = pd.DataFrame(compMoney)

In [764]:
df = df.explode('offices')
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
cleanData = pd.concat([df,dfOfficeData], axis=1)

In [765]:
cleanData = cleanData.drop(columns=["offices", "description", "address1", "address2", "zip_code", "state_code", "country_code"])
cleanData = cleanData.dropna()

In [766]:
atlanta = cleanData[cleanData['city'].str.contains("Atlanta")]
display(atlanta)
len(atlanta)

,name,total_money_raised,city,latitude,longitude
577,HowStuffWorks,$75M,Atlanta,37.090240,-95.712891
851,Zipcar,$60.7M,NW Atlanta,33.786263,-84.387794
949,Vertical Acuity,$6.08M,Atlanta,33.865682,-84.470944
1086,Google,$555M,Atlanta,33.781466,-84.387519
1442,EnerTech Environmental,$42M,Atlanta,33.772610,-84.350741
1505,Silverpop,$40M,Atlanta,33.882526,-84.465515
1508,Dayforce,$40M,Atlanta,33.887619,-84.710865
1554,Purewire,$4.75M,Atlanta,33.852384,-84.380568
1555,Purewire,$4.75M,Atlanta,33.852384,-84.380568
1824,Vitrue,$33M,Atlanta,33.756806,-84.392198


20

In [767]:
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
        

atlanta["location"] = atlanta[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
atlanta[["latitude","longitude","location"]]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,latitude,longitude,location
577,37.090240,-95.712891,"{'type': 'Point', 'coordinates': [-95.712891, ..."
851,33.786263,-84.387794,"{'type': 'Point', 'coordinates': [-84.3877936,..."
949,33.865682,-84.470944,"{'type': 'Point', 'coordinates': [-84.4709443,..."
1086,33.781466,-84.387519,"{'type': 'Point', 'coordinates': [-84.387519, ..."
1442,33.772610,-84.350741,"{'type': 'Point', 'coordinates': [-84.3507414,..."
1505,33.882526,-84.465515,"{'type': 'Point', 'coordinates': [-84.4655151,..."
1508,33.887619,-84.710865,"{'type': 'Point', 'coordinates': [-84.710865, ..."
1554,33.852384,-84.380568,"{'type': 'Point', 'coordinates': [-84.380568, ..."
1555,33.852384,-84.380568,"{'type': 'Point', 'coordinates': [-84.380568, ..."
1824,33.756806,-84.392198,"{'type': 'Point', 'coordinates': [-84.392198, ..."


## Vamos ahora hacer Web Scrapping para conseguir las coordenadas del aeropuerto de Atlanta

In [768]:
url = 'https://es.wikipedia.org/wiki/Aeropuerto_Internacional_Hartsfield-Jackson'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [769]:
airportLat = soup.select('span.latitude')
airportLatitud = [a.text for a in airportLat]
airportLatitud = airportLatitud[1].split(',')
airportLatitud = float(airportLatitud[0])
airportLatitud

33.636666666667

In [770]:
airportLon = soup.select('span.longitude')
airportLongitud = float([a.text for a in airportLon][1])
airportLongitud

-84.428055555556

In [771]:
#airportName = soup.select('#firstHeading')
#airportName

In [772]:
aeropuerto = pd.DataFrame({'longitude': airportLongitud, 'latitude': airportLatitud}, index=[0])
aeropuerto

,longitude,latitude
0,-84.428056,33.636667


In [773]:
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
        

aeropuerto["location"] = aeropuerto[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
aeropuerto[["latitude","longitude","location"]]

,latitude,longitude,location
0,33.636667,-84.428056,"{'type': 'Point', 'coordinates': [-84.42805555..."


In [774]:
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

atlantaUsa = geocode("Atlanta")
atlantaUsa

{'type': 'Point', 'coordinates': [-84.39015, 33.77729]}

In [775]:
def withGeoQuery(location,maxDistance=8000,minDistance=0,field="location"):
    return {
       field: {
         "$near": {
           "$geometry": location if type(location)==dict else geocode(location),
           "$maxDistance": maxDistance,
           "$minDistance": minDistance
         }
       }
    }

In [776]:
from pprint import pprint 

query = withGeoQuery(atlantaUsa)
pprint(query)

{'location': {'$near': {'$geometry': {'coordinates': [-84.39015, 33.77729],
                                      'type': 'Point'},
                        '$maxDistance': 8000,
                        '$minDistance': 0}}}


## Vamos ahora con los Starbucks

In [777]:
client = MongoClient("mongodb://localhost/starbucks")
db = client.get_database()

In [778]:
starbucksAtlanta = db.starbucks.find({"$and":[{"City":{"$eq":"Atlanta"}},{"Brand":{"$eq":"Starbucks"}}]},{'_id':0})
starbucksAtlanta = pd.DataFrame(starbucksAtlanta)

In [779]:
starbAtlanta = starbucksAtlanta.drop(columns=["Store Number", "Store Name", "State/Province", "Country", "Postcode", "Phone Number", "Timezone"])
starbAtlanta

,Brand,Ownership Type,Street Address,City,Longitude,Latitude
0,Starbucks,Company Owned,3520 Ashford Dunwoody Road,Atlanta,-84.33,33.89
1,Starbucks,Licensed,"2400 N Druid Hills Rd NE, Abernathy Square",Atlanta,-84.33,33.83
2,Starbucks,Company Owned,6681 K Roswell Rd,Atlanta,-84.38,33.94
3,Starbucks,Licensed,4484 Peachtree Rd NE,Atlanta,-84.33,33.87
4,Starbucks,Company Owned,"650 Ponce De Leon Avenue, Suite 700",Atlanta,-84.37,33.77
5,Starbucks,Company Owned,867 Peachtree St NE,Atlanta,-84.38,33.78
6,Starbucks,Licensed,1380 Oxflord Rd,Atlanta,-84.33,33.79
7,Starbucks,Company Owned,1152 Cumberland Mall,Atlanta,-84.47,33.88
8,Starbucks,Company Owned,"240 Peachtree Street NW, Cumberland Mall",Atlanta,-84.39,33.76
9,Starbucks,Licensed,266 4th St NW,Atlanta,-84.39,33.78


In [780]:
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
        

atlanta["location"] = atlanta[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
atlanta[["latitude","longitude","location"]]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,latitude,longitude,location
577,37.090240,-95.712891,"{'type': 'Point', 'coordinates': [-95.712891, ..."
851,33.786263,-84.387794,"{'type': 'Point', 'coordinates': [-84.3877936,..."
949,33.865682,-84.470944,"{'type': 'Point', 'coordinates': [-84.4709443,..."
1086,33.781466,-84.387519,"{'type': 'Point', 'coordinates': [-84.387519, ..."
1442,33.772610,-84.350741,"{'type': 'Point', 'coordinates': [-84.3507414,..."
1505,33.882526,-84.465515,"{'type': 'Point', 'coordinates': [-84.4655151,..."
1508,33.887619,-84.710865,"{'type': 'Point', 'coordinates': [-84.710865, ..."
1554,33.852384,-84.380568,"{'type': 'Point', 'coordinates': [-84.380568, ..."
1555,33.852384,-84.380568,"{'type': 'Point', 'coordinates': [-84.380568, ..."
1824,33.756806,-84.392198,"{'type': 'Point', 'coordinates': [-84.392198, ..."
